Description: stiches sensors and potholes files together into a new combined file using a given time window

In [21]:
import pandas as pd
pd.options.display.max_rows = 100
pd.set_option('display.precision', 10)  #for displaying timestamps

In [22]:
sensorsFilePath = 'data/Final_Route/t6-random_sensors.csv'
potholesFilePath = 'data/Final_Route/t6-random_potholes.csv'
combinedFilePath = 'data/Final_Route/t6-random_combined.csv'
sensorsDF = pd.read_csv(sensorsFilePath)
potholesDF = pd.read_csv(potholesFilePath)

In [23]:
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ
0,1.4930035626e+09,40.4475257722,-79.9441930614,0.0,0.2176666260,-0.9800415039,0.1583404541,0.0329595480,0.0487339837,0.1078993031
1,1.4930035627e+09,40.4475257722,-79.9441930614,0.0,0.0113525391,-0.9469299316,0.2120971680,0.0235993357,-0.0873756522,0.0175265294
2,1.4930035629e+09,40.4475257722,-79.9441930614,0.0,0.0236511230,-0.9569549561,0.2081146240,-0.0095865810,0.0053814496,0.0030866037
3,1.4930035631e+09,40.4475257822,-79.9441930619,0.0,0.0610809326,-0.9560852051,0.2014617920,-0.0148543136,0.0011598067,-0.0033537188
4,1.4930035633e+09,40.4475257822,-79.9441930619,0.0,0.0701599121,-0.9564666748,0.1945037842,-0.0127994185,-0.0009803096,0.0009376990


In [24]:
potholesDF.head()

,timestamp
0,1.4930036857e+09
1,1.4930036867e+09
2,1.4930036926e+09
3,1.4930037090e+09
4,1.4930037140e+09


#### Group points into time intervals with combined sensor+pothole data

In [71]:
# Time window (2 seconds) in number of points
# 5 points = 1 second
window = 10

intervalsDF = pd.DataFrame(columns=['avgSpeed', 'maxAccelX', 'maxAccelY', 
                                    'maxAccelZ', 'maxGyroX', 'maxGyroY', 
                                    'maxGyroZ', 'pothole'])

for i in xrange(0, len(sensorsDF), window):
    intervalIndex = len(intervalsDF)
    interval = sensorsDF[i:i+10]
    intervalStart = list(interval['timestamp'])[0]
    intervalEnd = list(interval['timestamp'])[-1]
    avgSpeed = interval['speed'].mean()
    maxAccelX = interval['accelerometerX'].max()
    maxAccelY = interval['accelerometerY'].max()
    maxAccelZ = interval['accelerometerZ'].max()
    maxGyroX = interval['gyroX'].max()
    maxGyroY = interval['gyroY'].max()
    maxGyroZ = interval['gyroZ'].max()
    pothole = intervalHasPothole(intervalStart, intervalEnd, potholesDF)
    intervalSummary = [avgSpeed, maxAccelX, maxAccelY, maxAccelZ, 
                       maxGyroX, maxGyroY, maxGyroZ, pothole]
    intervalsDF.loc[intervalIndex] = intervalSummary

def intervalHasPothole(intervalStart, intervalEnd, potholesDF):
    potholeTimestamps = potholesDF['timestamp']
    for index, potholeTime in potholeTimestamps.iteritems():
        if intervalStart <= potholeTime and potholeTime <= intervalEnd:
            return True
    return False

intervalsDF

,avgSpeed,maxAccelX,maxAccelY,maxAccelZ,maxGyroX,maxGyroY,maxGyroZ,pothole
0,0.0000000000,0.2176666260,-0.9469299316,0.2120971680,0.0329595480,0.0487339837,0.1078993031,False
1,0.0479999989,0.0785064697,-0.9331359863,0.2717742920,-0.0031795480,0.0096606169,0.0072680329,False
2,0.2259999931,0.0674438477,-0.9461822510,0.2098083496,-0.0063337960,0.0054416371,0.0020445088,False
3,0.0569999993,0.1011352539,-0.9107055664,0.3683624268,0.0032314797,0.0086475504,0.0052911685,False
4,1.2889999986,0.1100006104,-0.8900909424,0.3652343750,-0.0053862433,0.0597397633,0.0158157148,False
5,4.3520001411,0.1663055420,-0.7996215820,0.4277496338,0.0628141164,0.0043729105,0.1021897521,False
6,7.8339998245,0.1202239990,-0.8634338379,0.4615631104,0.1123204169,-0.0037428066,0.2536003138,False
7,9.5299999237,0.1236419678,-0.8696594238,0.3550872803,0.0191686346,0.0360876970,0.0652932531,False
8,11.7229999542,0.1282501221,-0.8537445068,0.3307647705,0.0598220549,0.0549247680,0.0783486014,False
9,12.4870003700,0.2998199463,-0.9181671143,0.3240203857,0.0201904895,0.2154289595,0.0333006989,False


In [42]:
potholeLabels = []   #indicator for pothole at each timestamp
sensorIndex = 0
sensorTimestamp = sensorsDF['timestamp'][sensorIndex]
halfWindow = window/2

#Adds pothole indicators based on time and window
for index, potholeTimestamp in potholesDF['timestamp'].iteritems():
    while sensorTimestamp <= (potholeTimestamp + halfWindow):
        if (potholeTimestamp - halfWindow) <= sensorTimestamp:
            potholeLabels.append(True)
        else:
            potholeLabels.append(False)
        sensorIndex += 1
        sensorTimestamp = sensorsDF['timestamp'][sensorIndex]

#No potholes for remainder of data 
#(when sensorsDF larger than potholesDF)
remainingIndices = len(sensorsDF) - sensorIndex
remainingPotholeLabels = [False] * remainingIndices
potholeLabels.extend(remainingPotholeLabels)

In [43]:
sensorsDF['potholes'] = potholeLabels
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
0,1.4930035626e+09,40.4475257722,-79.9441930614,0.0,0.2176666260,-0.9800415039,0.1583404541,0.0329595480,0.0487339837,0.1078993031,False
1,1.4930035627e+09,40.4475257722,-79.9441930614,0.0,0.0113525391,-0.9469299316,0.2120971680,0.0235993357,-0.0873756522,0.0175265294,False
2,1.4930035629e+09,40.4475257722,-79.9441930614,0.0,0.0236511230,-0.9569549561,0.2081146240,-0.0095865810,0.0053814496,0.0030866037,False
3,1.4930035631e+09,40.4475257822,-79.9441930619,0.0,0.0610809326,-0.9560852051,0.2014617920,-0.0148543136,0.0011598067,-0.0033537188,False
4,1.4930035633e+09,40.4475257822,-79.9441930619,0.0,0.0701599121,-0.9564666748,0.1945037842,-0.0127994185,-0.0009803096,0.0009376990,False


In [44]:
sensorsDF[sensorsDF['potholes'] == True]

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
614,1.4930036853e+09,40.4532104320,-79.9475353756,9.9799995422,0.3191528320,-1.2776489258,0.1109619141,0.0286076764,0.2178721435,-0.0258584952,True
615,1.4930036855e+09,40.4532104320,-79.9475353756,9.9799995422,-0.2827911377,-0.8435974121,0.4779815674,0.0714238499,-0.0344826098,0.1308983623,True
616,1.4930036857e+09,40.4532104320,-79.9475353756,9.9799995422,-0.7182464600,-1.3249511719,0.6968536377,0.1267022857,0.2937772946,-0.3033319176,True
617,1.4930036859e+09,40.4532104320,-79.9475353756,9.9799995422,-0.1927032471,-0.8647766113,0.3640594482,0.0198575944,0.3210717663,-0.2629139207,True
618,1.4930036861e+09,40.4532104320,-79.9475353756,9.9799995422,0.0528717041,-1.0030517578,0.3737792969,-0.1635183572,0.2074003278,-0.1546779940,True
619,1.4930036863e+09,40.4531762056,-79.9476506348,10.9399995804,0.1708221436,-0.6574707031,0.3615417480,0.1304916976,-0.2859073873,0.1129601082,True
620,1.4930036865e+09,40.4531762056,-79.9476506348,10.9399995804,-0.3746490479,-1.2853546143,0.6541442871,-0.4907811741,0.6253592261,-0.6783534686,True
621,1.4930036867e+09,40.4531762056,-79.9476506348,10.9399995804,0.4201660156,-0.9645080566,0.1910705566,-0.0145075700,0.1032670007,-0.0505270901,True
622,1.4930036869e+09,40.4531762056,-79.9476506348,10.9399995804,0.0760345459,-0.9176940918,0.3105926514,-0.0381966543,0.0427868787,-0.0644743310,True
623,1.4930036871e+09,40.4531762056,-79.9476506348,10.9399995804,0.0223083496,-0.9021759033,0.3123168945,-0.1000493695,-0.0291240634,0.0576925913,True


In [45]:
sensorsDF.to_csv(combinedFilePath)